# 🔥 InvEye YOLO11 Training Notebook
## Employee Tracking + Petrol Pump + Fire/Fight Detection

---

### ⚠️ Before Running:
1. **Runtime → Change runtime type → T4 GPU**
2. Run cells **in order**
3. Get Roboflow API key from https://app.roboflow.com/settings/api

## 1️⃣ Setup

In [ ]:
!pip install -q ultralytics roboflow
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NOT AVAILABLE - Change runtime to GPU!'}")

In [ ]:
from ultralytics import YOLO
from google.colab import drive, files
import os, shutil, yaml
from pathlib import Path

drive.mount('/content/drive')
os.makedirs('/content/inveye', exist_ok=True)
os.chdir('/content/inveye')
print('✅ Setup complete')

## 2️⃣ Download Datasets (Choose ONE method)

### Option A: Roboflow (Recommended - needs API key)

In [ ]:
#@title Enter your Roboflow API Key { display-mode: "form" }
ROBOFLOW_API_KEY = "" #@param {type:"string"}

if ROBOFLOW_API_KEY:
    from roboflow import Roboflow
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    
    # Download fire dataset
    print('📥 Downloading fire detection dataset...')
    try:
        ds = rf.workspace('roboflow-100').project('fire-smoke-detection').version(1).download('yolov8', location='./datasets/fire')
        print(f'✅ Fire dataset downloaded')
    except:
        try:
            ds = rf.workspace('roboflow-universe-projects').project('fire-and-smoke-detection-nyvru').version(1).download('yolov8', location='./datasets/fire')
            print(f'✅ Fire dataset downloaded (alt)')
        except Exception as e:
            print(f'⚠️ Fire: {e}')
    
    # Download PPE dataset
    print('📥 Downloading PPE dataset...')
    try:
        ds = rf.workspace('roboflow-universe-projects').project('hard-hat-sample-ds-uddk0').version(1).download('yolov8', location='./datasets/ppe')
        print(f'✅ PPE dataset downloaded')
    except Exception as e:
        print(f'⚠️ PPE: {e}')
else:
    print('⚠️ No API key - use Option B or C below')

### Option B: Direct Download (No API key needed)

In [ ]:
# Download pre-made fire/smoke dataset from GitHub
print('📥 Downloading fire dataset from GitHub...')
!git clone --depth 1 https://github.com/gaiasd/DFireDataset.git ./datasets/dfire 2>/dev/null || echo 'Already exists'

# Check what we have
!ls -la ./datasets/ 2>/dev/null || echo 'No datasets yet'
print('\n✅ Done - run next cell to verify')

### Option C: Use COCO subset (Always works)

In [ ]:
# Use COCO128 - small dataset for testing
print('📥 Downloading COCO128 (small test dataset)...')
from ultralytics.data.utils import download
download('https://ultralytics.com/assets/coco128.zip', dir='./datasets')
print('✅ COCO128 downloaded')

## 3️⃣ Verify Dataset

In [ ]:
# Check dataset structure
print('📂 Dataset contents:')
!find ./datasets -type d -name 'images' | head -10
!find ./datasets -type d -name 'train' | head -10

# Count images
import glob
images = glob.glob('./datasets/**/*.jpg', recursive=True) + glob.glob('./datasets/**/*.png', recursive=True)
print(f'\n📊 Total images found: {len(images)}')

if len(images) == 0:
    print('\n❌ No images found! Please run Option B or C above.')
else:
    print('\n✅ Dataset ready!')

## 4️⃣ Create Dataset Config

In [ ]:
# Find the dataset YAML that exists
import glob

yaml_files = glob.glob('./datasets/**/*.yaml', recursive=True)
print('Found YAML configs:', yaml_files)

if yaml_files:
    DATASET_YAML = yaml_files[0]
    print(f'\n✅ Using: {DATASET_YAML}')
else:
    # Create one for COCO128
    DATASET_YAML = './datasets/coco128/coco128.yaml'
    if not os.path.exists(DATASET_YAML):
        # Use ultralytics built-in COCO128
        DATASET_YAML = 'coco128.yaml'
    print(f'Using built-in: {DATASET_YAML}')

print(f'\n📄 Dataset config: {DATASET_YAML}')

## 5️⃣ Download YOLO11 Model

In [ ]:
!wget -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11s.pt -O yolo11s.pt
print(f'✅ YOLO11s downloaded: {os.path.getsize("yolo11s.pt")/1e6:.1f} MB')

## 6️⃣ Train Model 🚀

In [ ]:
#@title Training Settings { display-mode: "form" }
EPOCHS = 50 #@param {type:"integer"}
BATCH_SIZE = 16 #@param {type:"integer"}
IMG_SIZE = 640 #@param {type:"integer"}

print(f'Training: {EPOCHS} epochs, batch {BATCH_SIZE}, img {IMG_SIZE}')

In [ ]:
# Train!
model = YOLO('yolo11s.pt')

print('🚀 Starting training...\n')
results = model.train(
    data=DATASET_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=0,
    project='runs',
    name='inveye',
    half=True,
    patience=20,
    plots=True,
)
print('\n✅ Training complete!')

## 7️⃣ Download Model

In [ ]:
# Save to Drive
BEST = 'runs/inveye/weights/best.pt'
DRIVE = '/content/drive/MyDrive/InvEye_Models'
os.makedirs(DRIVE, exist_ok=True)

if os.path.exists(BEST):
    shutil.copy(BEST, f'{DRIVE}/inveye_yolo11_best.pt')
    print(f'✅ Saved to Google Drive: {DRIVE}')
    files.download(BEST)
else:
    print('❌ Model not found - training may have failed')

## ✅ Done!

**Next steps:**
1. Download model from Google Drive
2. Copy to Jetson: `scp best.pt nvidia@<IP>:~/inveye/`
3. Export on Jetson: `python3 export_to_deepstream.py --model best.pt`
4. Run: `python3 inveye_multi_camera.py`